## Summary

In this notebook we load a network trained to reconstruct protein sequences and use this network to design sequences that fit the geometry of CATH domain [1n5uA03](http://www.cathdb.info/version/latest/domain/1n5uA03).

----

## Imports

In [1]:
import importlib
import logging
import subprocess
from pathlib import Path

import ipywidgets as widgets
import nglview
from IPython.display import HTML, display, set_matplotlib_formats
from ipywidgets import Layout

import proteinsolver
import proteinsolver.dashboard
import torch
from kmbio import PDB

_ColormakerRegistry()

RDKit WARNING: [21:54:18] Enabling RDKit 2019.09.3 jupyter extensions


In [2]:
%matplotlib agg

set_matplotlib_formats("svg")

In [3]:
DEBUG = True

In [4]:
if DEBUG:
    %load_ext autoreload
    %autoreload 2

## Properties

In [5]:
NOTEBOOK_NAME = "30_design_dashboard"

NOTEBOOK_PATH = Path(NOTEBOOK_NAME).resolve()
NOTEBOOK_PATH.mkdir(exist_ok=True)

In [6]:
NOTEBOOK_PATH

PosixPath('/home/kimlab1/strokach/workspace/proteinsolver/notebooks/30_design_dashboard')

In [7]:
JUPYTER_DATA_PATH = Path(subprocess.check_output(["jupyter", "--data-dir"], universal_newlines=True).strip()).resolve(strict=True)

In [8]:
JUPYTER_DATA_PATH

PosixPath('/home/strokach/.local/share/jupyter')

In [9]:
STATIC_DATA_PATH = JUPYTER_DATA_PATH.joinpath("voila", "templates", "mytemplate", "static").resolve(strict=True)

In [10]:
STATIC_DATA_PATH

PosixPath('/home/strokach/.local/share/jupyter/voila/templates/mytemplate/static')

In [11]:
UNIQUE_ID = "191f05de"

In [12]:
BEST_STATE_FILES = {
    #
    "191f05de": "protein_train/191f05de/e53-s1952148-d93703104.state"
}

In [13]:
LOGGER = logging.getLogger(__name__)

In [14]:
global_state = proteinsolver.dashboard.global_state
global_state.output_folder = STATIC_DATA_PATH

## Custom CSS

In [15]:
HTML("""\
<style>
h1 {
    margin-top: -20px !important;
}

.myheading {
    font-size: large;
}

.mytext > .widget-label {
    font-family: monospace;
    font-size: small;
    width: 60px;
}

.mysequence > .widget-label {
    line-height: 1rem;
    min-height: calc( var(--jp-widgets-inline-height) + 12);
}

/*
.mynglout {
    z-index: -100
}
*/

.biojs_msa_labelblock {
    overflow: hidden !important;
}

.ui-resizable {
    height: 250px !important;
}

img {
    max-width: 100% !important;
}
</style>
""")

## Load model

In [16]:
Net = importlib.import_module(f"protein_train.{UNIQUE_ID}.model").Net

In [17]:
global_state.net_class = Net
global_state.state_file = BEST_STATE_FILES[UNIQUE_ID]
global_state.net_kwargs = {"x_input_size": 21, "adj_input_size": 2, "hidden_size": 128, "output_size": 20}

## Design pipeline

### Load protein sequence and geometry

In [18]:
structure = PDB.load(Path(proteinsolver.__path__[0]).resolve(strict=True).joinpath("data", "inputs", "1n5uA03.pdb"))
proteinsolver.dashboard.load_structure(structure)

### Test ProteinSolverProcess

In [19]:
data = global_state.data
data.x = torch.tensor(
    [proteinsolver.utils.AMINO_ACID_TO_IDX[aa] for aa in global_state.target_sequence],
    dtype=torch.long,
)

In [20]:
global_state.proteinsolver_process = proteinsolver.dashboard.ProteinSolverProcess(
    global_state.net_class,
    global_state.state_file,
    global_state.data,
    num_designs=2,
    net_kwargs=global_state.net_kwargs,
)

In [21]:
if DEBUG:
    global_state.proteinsolver_process.start()

In [22]:
if DEBUG:
    global_state.generated_sequences = [global_state.proteinsolver_process.output_queue.get() for _ in range(2)]

In [23]:
if DEBUG:
    print(global_state.generated_sequences)

[MSASeq(id=0, name='gen-00001', seq='GICVVAQNILATDWLGERFPQASDVDMLELSAIYDLVLTECYVKDSFTCNAHQAKLVGHVCKSQDLFSPDLQECCKIPEGIVSACLATTNIN', ref=False, proba=0.26656419038772583, logproba=-1.9861204624176025), MSASeq(id=1, name='gen-00002', seq='KIGFQALQKLAVERMTDRFPYASIAEVLHLQHVLQVVFYCCQVCDAQQCNVCQSLVAAYICMNLSQISNDLASCCNIPVNKLNDCIAYCSFE', ref=False, proba=0.2731874883174896, logproba=-1.9825528860092163)]


In [24]:
if DEBUG:
    global_state.proteinsolver_process.join()

## Dashboard

### Structure view

In [25]:
ngl_stage = nglview.NGLWidget(
#     height="300px",
#     width="300px",
)

In [26]:
ngl_stage_out = widgets.Output(
    layout=widgets.Layout(width="20%", margin="0 0 0 0"))
ngl_stage_out.add_class("mynglout")

with ngl_stage_out:
    display(ngl_stage)

### Distance matrix view

In [27]:
distance_matrix_out = widgets.Output()

### Info panel view

In [28]:
target_selection_out = widgets.Output()

In [29]:
sequence_generation_out = widgets.Output()

In [30]:
load_structure_button = proteinsolver.dashboard.create_load_structure_button(
    ngl_stage, distance_matrix_out, target_selection_out, sequence_generation_out
)

In [32]:
load_distance_matrix_button = proteinsolver.dashboard.create_load_distance_matrix_button(
    ngl_stage, distance_matrix_out, target_selection_out, sequence_generation_out
)

In [33]:
load_example_buttons = proteinsolver.dashboard.create_load_example_buttons(
    ngl_stage, distance_matrix_out, target_selection_out, sequence_generation_out
)

In [35]:
load_structure_out = widgets.Output(
    layout=widgets.Layout(width="60%", margin="0px 0px 0px 0px"))

with load_structure_out:
    display(HTML('<p class="myheading">1. Load a reference protein structure</p>'))
    display(HTML('<p>Upload the distance matrix or the structure of a reference protein. An example distance matrix, corresponding to the protein <code>1n5uA03</code>, can be found <a href="voila/files/1n5uA03-distance-matrix.txt">here</a>. </p>'))
    display(widgets.HBox([load_structure_button, load_distance_matrix_button]))
    display(HTML('<p style="margin-top: 16px">Alternatively, try one of the examples provided below:</p>'))
    display(load_example_buttons)

# Generate protein sequences matching a predetermined geometry

In [36]:
out = widgets.VBox(
    [
        widgets.HBox(
            [
                load_structure_out,
                ngl_stage_out,
                widgets.HBox(
                    [distance_matrix_out],
                    layout=widgets.Layout(
                        width="19%",
                        margin="0 0 0 1%",
                        flex_flow="col nowrap",
                        justify_content="space-around",
                        align_items="center"
                    ),
                ),
            ],
            layout=widgets.Layout(
                width="auto",
                margin="0 0 0 0",
#                 flex_flow="row nowrap",
#                 justify_content="space-between",
            ),
        ),
        target_selection_out,
        sequence_generation_out,
    ]
)

In [37]:
display(out)

In [ ]:
# proteinsolver.dashboard.on_run_ps_button_clicked(run_ps_button, num_designs_field)

In [ ]:
display(HTML("""\
<hr>
<p>Running into issues? Please send an email to <a href="help@proteinsolver.org">help@proteinsolver.org</a>.
<br>
<em>This website works best using the latest versions of Firefox or Chrome web browsers.</em>
</p>
"""))